# 1、买卖股票问题

每天都有三种「选择」：买入、卖出、无操作，我们用 buy, sell, rest 表示这三种选择。但问题是，并不是每天都可以任意选择这三种选择的，因为 

sell 必须在 buy 之后，buy 必须在 sell 之后。那么 rest 操作还应该分两种状态，一种是 buy 之后的 rest（持有了股票），一种是 sell 之后的 

rest（没有持有股票）。而且别忘了，我们还有交易次数 k 的限制，就是说你 buy 还只能在 k > 0 的前提下操作。

dp[i][k][0 or 1]

0 <= i <= n-1, 1 <= k <= K

n 为天数，大 K 为最多交易数

此问题共 n × K × 2 种状态，全部穷举就能搞定。

for 0 <= i < n:

    for 1 <= k <= K:
    
        for s in {0, 1}:
        
            dp[i][k][s] = max(buy, sell, rest)
            

dp[i][k][0] = max(dp[i-1][k][0], dp[i-1][k][1] + prices[i])
              
              max(   选择 rest  ,           选择 sell      )

解释：今天我没有持有股票，有两种可能：

要么是我昨天就没有持有，然后今天选择 rest，所以我今天还是没有持有；

要么是我昨天持有股票，但是今天我 sell 了，所以我今天没有持有股票了。

dp[i][k][1] = max(dp[i-1][k][1], dp[i-1][k-1][0] - prices[i])

              max(   选择 rest  ,           选择 buy         )

解释：今天我持有着股票，有两种可能：

要么我昨天就持有着股票，然后今天选择 rest，所以我今天还持有着股票；

要么我昨天本没有持有，但今天我选择 buy，所以今天我就持有股票了。

现在，我们已经完成了动态规划中最困难的一步：状态转移方程。如果之前的内容你都可以理解，那么你已经可以秒杀所有问题了，只要套这个框架就行了。不

过还差最后一点点，就是定义 base case，即最简单的情况。

dp[-1][k][0] = 0

解释：因为 i 是从 0 开始的，所以 i = -1 意味着还没有开始，这时候的利润当然是 0 。
dp[-1][k][1] = -infinity

解释：还没开始的时候，是不可能持有股票的，用负无穷表示这种不可能。
dp[i][0][0] = 0

解释：因为 k 是从 1 开始的，所以 k = 0 意味着根本不允许交易，这时候利润当然是 0 。
dp[i][0][1] = -infinity

解释：不允许交易的情况下，是不可能持有股票的，用负无穷表示这种不可能。


# 重点来了

* base case：

dp[-1][k][0] = dp[i][0][0] = 0

dp[-1][k][1] = dp[i][0][1] = -infinity

* 状态转移方程：

dp[i][k][0] = max(dp[i-1][k][0], dp[i-1][k][1] + prices[i])

dp[i][k][1] = max(dp[i-1][k][1], dp[i-1][k-1][0] - prices[i])


# 1、k = 1

# [leetcode 121]

给定一个数组，它的第 i 个元素是一支给定股票第 i 天的价格。

如果你最多只允许完成一笔交易（即买入和卖出一支股票），设计一个算法来计算你所能获取的最大利润。

注意你不能在买入股票前卖出股票。



In [3]:
# 1、动态规划
def maxProfit(prices):
    """
    :type prices: List[int]
    :rtype: int
    """
    n = len(prices)
    dp = [[0]*2 for _ in range(n)]
    dp[0][0] = 0
    dp[0][1] = -prices[0]
    for i in range(1, n):
        dp[i][0] = max(dp[i-1][0], dp[i-1][1]+prices[i])
        dp[i][1] = max(dp[i-1][1], dp[i-1][0]-prices[i])
    return dp[n-1][0]


# 2、暴力解法

def maxProfit(prices):
    """
    :type prices: List[int]
    :rtype: int
    """
    maxProfit = 0
    for i in range(len(prices)-1):
        for j in range(i+1,len(prices)):
            profit =  prices[j]-prices[i]
            if profit > maxProfit:
                maxProfit =profit
    return maxProfit


# 3、一次遍历：
#  我们可以维持两个变量——minprice 和 maxprofit，它们分别对应迄今为止所得到的最小的谷值和最大的利润
#  (卖出价格与最低价格之间的最大差值）。
def maxProfit(prices):
    """
    :type prices: List[int]
    :rtype: int
    """
    import sys
    minPrices = sys.maxint
    maxProfit = 0
    for i in range(len(prices)):
        if prices[i] < minPrices:
            minPrices = prices[i]
        elif prices[i] - minPrices > maxProfit:
            maxProfit = prices[i] - minPrices
    
    return maxProfit


# 4、同3

def maxProfit(prices):
    """
    :type prices: List[int]
    :rtype: int
    """
    if len(prices)<=1:
        return 0
    buy_price = prices[0]
    max_profit = 0
    for i in range(1, len(prices)):
        buy_price = min(buy_price, prices[i])
        max_profit = max(max_profit, prices[i]-buy_price)
    return max_profit



# k = 2
# [leetcode 123]

给定一个数组，它的第 i 个元素是一支给定的股票在第 i 天的价格。

设计一个算法来计算你所能获取的最大利润。你最多可以完成 两笔 交易。

注意: 你不能同时参与多笔交易（你必须在再次购买前出售掉之前的股票）。



In [28]:
class Solution(object):
    def maxProfit(self, prices):
        """
        :type prices: List[int]
        :rtype: int
        """
        n = len(prices)
        if n < 2:
            return 0
        pre_max_profit = [0 for i in range(n)]
        pro_max_profit = [0 for i in range(n)]
        max_profit = 0
        pre_min_buy = prices[0]
        
        for i in range(1, n):
            pre_min_buy = min(pre_min_buy, prices[i])
            pre_max_profit[i] = max(pre_max_profit[i-1], prices[i]-pre_min_buy)
        print(pre_max_profit)

            
        pro_max_sell = prices[n-1]
        for k in range(n-2, -1,-1):
            pro_max_sell = max(prices[k], pro_max_sell)
            pro_max_profit[k] = max(pro_max_profit[k+1], pro_max_sell-prices[k])
        
        print(pro_max_profit)

        
        for j in range(0,n):
            max_profit = max(max_profit, pre_max_profit[j]+pro_max_profit[j])
        return max_profit

    
if __name__ == '__main__':
    
    prices = [3,3,5,0,0,3,1,4]
    max_profit = Solution().maxProfit(prices)
    print(max_profit)
    

[0, 0, 2, 2, 2, 3, 3, 4]
[4, 4, 4, 4, 4, 3, 3, 0]
6


# 70、爬楼梯

假设你正在爬楼梯。需要 n 阶你才能到达楼顶。

每次你可以爬 1 或 2 个台阶。你有多少种不同的方法可以爬到楼顶呢？

注意：给定 n 是一个正整数。

示例 1：

输入： 2

输出： 2

解释： 有两种方法可以爬到楼顶。

1.  1 阶 + 1 阶
2.  2 阶

示例 2：

输入： 3

输出： 3

解释： 有三种方法可以爬到楼顶。

1.  1 阶 + 1 阶 + 1 阶

2.  1 阶 + 2 阶

3.  2 阶 + 1 阶


In [5]:
# 斐波那切数列：f(n)=f(n-1)+f(n-2)

class Solution(object):
    def climbStairs(self, n):
        """
        :type n: int
        :rtype: int
        """
        if n < 1:
            return 0
        a, b = 0, 1
        for i in range(1, n+1):
            a, b = b, a+b
        return b


# 198、打家劫舍

你是一个专业的小偷，计划偷窃沿街的房屋。每间房内都藏有一定的现金，影响你偷窃的唯一制约因素就是相邻的房屋装有相互连通的防盗系统，如果两间相邻

的房屋在同一晚上被小偷闯入，系统会自动报警。

给定一个代表每个房屋存放金额的非负整数数组，计算你在不触动警报装置的情况下，能够偷窃到的最高金额。

示例 1:

输入: [1,2,3,1]

输出: 4

解释: 偷窃 1 号房屋 (金额 = 1) ，然后偷窃 3 号房屋 (金额 = 3)。
     
     偷窃到的最高金额 = 1 + 3 = 4 。

示例 2:

输入: [2,7,9,3,1]

输出: 12

解释: 偷窃 1 号房屋 (金额 = 2), 偷窃 3 号房屋 (金额 = 9)，接着偷窃 5 号房屋 (金额 = 1)。
    
    偷窃到的最高金额 = 2 + 9 + 1 = 12 。


方法一：动态规划

考虑所有可能的抢劫方案过于困难。一个自然而然的想法是首先从最简单的情况开始。记：

f(k) = 从前 k 个房屋中能抢劫到的最大数额，A_i= 第 i 个房屋的钱数。

首先看 n = 1 的情况，显然 f(1) = A_1。

再看 n = 2，f(2) = max(A_1, A_2)。

对于 n = 3，有两个选项:

* 抢第三个房子，将数额与第一个房子相加。

* 不抢第三个房子，保持现有最大数额。

显然，你想选择数额更大的选项。于是，可以总结出公式：

f(k) = max(f(k – 2) + A_k, f(k – 1))

我们选择 f(–1) = f(0) = 0 为初始情况，这将极大地简化代码。

答案为 f(n)。可以用一个数组来存储并计算结果。不过由于每一步你只需要前两个最大值，两个变量就足够用了。


In [63]:
class Solution:
    def rob(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        r = [0]*(len(nums))
        if len(nums)==0:
            return 0
        if len(nums)==1:
            return nums[0]
        if len(nums)==2:
            return max(nums[0], nums[1])
        else:
            r[0] = nums[0]
            r[1] = max(nums[0], nums[1])
            for i in range(2, len(nums)):
                r[i] = max(r[i-1], r[i-2]+nums[i])
        return r[len(nums)-1]
    

            
if __name__ == '__main__':
    nums = [2,7,9,3,1]
    r = Solution().rob(nums)
    print(r)



12


# 53、最大子序和

给定一个整数数组 nums ，找到一个具有最大和的连续子数组（子数组最少包含一个元素），返回其最大和。

示例:

输入: [-2,1,-3,4,-1,2,1,-5,4],

输出: 6

解释: 连续子数组 [4,-1,2,1] 的和最大，为 6。

In [ ]:
# 方法一
class Solution:
    def maxSubArray(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        l = len(nums)
        i = 0
        sum = 0
        MaxSum = nums[0]
        while i < l:
            sum+=nums[i]
            if sum > MaxSum:
                MaxSum = sum
            if sum < 0:
                sum = 0
            i+=1
        return MaxSum


# 方法二    
class Solution(object):
    def maxSubArray(self, nums):
        """
        :type nums: List[int]
        :rtype: int
         """
        for i in range(1, len(nums)):
            nums[i]= nums[i] + max(nums[i-1], 0)
        return max(nums)


# 152、乘积最大子序列

给定一个整数数组 nums ，找出一个序列中乘积最大的连续子序列（该序列至少包含一个数）。

示例 1:

输入: [2,3,-2,4]

输出: 6

解释: 子数组 [2,3] 有最大乘积 6。

示例 2:

输入: [-2,0,-1]

输出: 0

解释: 结果不能为 2, 因为 [-2,-1] 不是子数组。


In [11]:
# 方法一：
class Solution(object):
    def maxProduct(self, nums):
        
        if len(nums) == 1: 
            return nums[0]
        
        pos_max, neg_max = 0, 0
        
        res = nums[0]
        for x in nums:
            curp = pos_max # 避免pos_max被修改，先存起来
            pos_max = max(x, max(x*pos_max, x*neg_max))
            neg_max = min(x, min(x*curp, x*neg_max))
            res = max(pos_max, res)
        
        return res



In [65]:
# 方法二：先计算从左到右的相乘的最大值，再计算从右到左的最大值；再将两组最大值相比

def maxProduct(A):
    B = A[::-1]
    for i in range(1, len(A)):
        A[i] *= A[i - 1] or 1
        B[i] *= B[i - 1] or 1
    print(A)
    print(B)
    return max(max(A),max(B)) 

if __name__ == '__main__':
    A = [2,3,-2,4]
    m = maxProduct(A)
    print(m)
    

[2, 6, -12, -48]
[4, -8, -24, -48]
6


# 322、零钱兑换

给定不同面额的硬币 coins 和一个总金额 amount。编写一个函数来计算可以凑成总金额所需的最少的硬币个数。如果没有任何一种硬币组合能组成总金额，返回 -1。

示例 1:

输入: coins = [1, 2, 5], amount = 11

输出: 3 

解释: 11 = 5 + 5 + 1

示例 2:

输入: coins = [2], amount = 3

输出: -1



## 思路一：动态规划

假设 f(n) 代表要凑齐金额为 n 所要用的最少硬币数量，那么有：

f(n) = min(f(n - c1), f(n - c2), ... f(n - cn)) + 1

其中 c1 ~ cn 为硬币的所有面额。

----

再具体解释一下这个公式吧，例如这个示例：

输入: coins = [1, 2, 5], amount = 11

输出: 3 

解释: 11 = 5 + 5 + 1

题目求的值为 f(11)，第一次选择硬币时我们有三种选择。

假设我们取面额为 1 的硬币，那么接下来需要凑齐的总金额变为 11 - 1 = 10，即 f(11) = f(10) + 1，这里的 +1 就是我们取出的面额为 1 的硬币。

同理，如果取面额为 2 或面额为 5 的硬币可以得到：

f(11) = f(9) + 1

f(11) = f(6) + 1

所以：

f(11) = min(f(10), f(9), f(6)) + 1

In [26]:
# 方法一
class Solution(object):
    def coinChange(self, coins, amount):
        """
        :type coins: List[int]
        :type amount: int
        :rtype: int
        """
        res = [0]*amount
        
        for i in range(1, amount+1):
            cost = float('inf')
            for c in coins:
                if i-c>=0:
                    cost = min(cost, res[i-c]+1)
            res[i]=cost
            
        if res[amount]==folat['inf']:
            return -1
        else:
            return res[amount]
        
        
# 同一
class Solution:
    def coinChange(self, coins, amount):

        dp = [float("inf")] * (amount+1)
        dp[0] = 0
        
        for i in range(1,amount+1):
            for coin in coins:
                if i - coin>= 0:
                    dp[i] = min(dp[i],dp[i-coin]+1)
        return dp[-1] if dp[-1]!= float("inf") else -1        

# 357、计算各个位数不同的数字个数

给定一个非负整数 n，计算各位数字都不同的数字 x 的个数，其中 0 ≤ x < 10n 。

示例:

输入: 2

输出: 91 

解释: 答案应为除去 11,22,33,44,55,66,77,88,99 外，在 [0,100) 区间内的所有数字。


思路一：动态规划

n=0: res[0] = 1

n=1: res[1]=10

n=2: res[2]=9 * 9+10=91= res[1]+(res[1]-res[0])*(10-1)  # 两位数第一位只能为1-9，第二位只能为非第一位的数，加上一位数的所有结果

n=3: res[3]=9 * 9 * 8+91=739=res[2]+(res[2]-res[1])*(10-2) # 三位数第一位只能为1-9，第二位只能为非第一位的数，第三位只能为非前两位的数，加上两位数的所有结果

n=4: res[4]=9 * 9 * 8 * 7+739=5275 = res[3]+(res[3]-res[2])*(10-3) # 同上推法

所以有： res[n] = res[n-1] + (res[n-1]-res[n-2])*(10-(n-1))



In [27]:
class Solution(object):
    def countNumbersWithUniqueDigits(self, n):
        """
        :type n: int
        :rtype: int
        """
        if n==0:
            return 1
        dp = [0]*(n+1)
        dp[0] = 1
        dp[1] = 10
        for i in range(2,n+1):
            dp[i] = dp[i-1]+(dp[i-1]-dp[i-2])*(10-(i-1))
        return dp[n]
    
    
class Solution(object):
    def countNumbersWithUniqueDigits(self, n):
        """
        :type n: int
        :rtype: int
        """   
        if not n:
            return 1
        res, muls = 10, 9
        for i in range(1, min(n, 10)):
            muls *= 10-i
            res += muls
        return res
    

# 466. 统计重复个数


定义由 n 个连接的字符串 s 组成字符串 S，即 S = [s,n]。例如，["abc", 3]=“abcabcabc”。

另一方面，如果我们可以从 s2 中删除某些字符使其变为 s1，我们称字符串 s1 可以从字符串 s2 获得。例如，“abc” 可以根据我们的定义从 “abdbec” 

获得，但不能从 “acbbe” 获得。

现在给出两个非空字符串 S1 和 S2（每个最多 100 个字符长）和两个整数 0 ≤ N1 ≤ 106 和 1 ≤ N2 ≤ 106。现在考虑字符串 S1 和 S2，其中S1=

[s1,n1]和S2=[s2,n2]。找出可以使[S2,M]从 S1 获得的最大整数 M。

示例：

输入：

s1 ="acb",n1 = 4

s2 ="ab",n2 = 2

返回：2



## 简单说就是统计S1中有多少个S2



In [32]:
# 暴力遍历法：不能温泉部通过测试样例

class Solution(object):
    def getMaxRepetitions(self, s1, n1, s2, n2):
        """
        :type s1: str
        :type n1: int
        :type s2: str
        :type n2: int
        :rtype: int
        """

        # index为s2的索引， num1当前使用了s1的个数， num2当前匹配的s2的个数
        index, num1, num2 = 0, 0, 0
        while num1 < n1:
            for i in range(len(s1)):
                if s1[i]==s2[index]:
                    if index == len(s2)-1:
                        index = 0
                        num2 += 1
                    else:
                        index += 1
            num1 += 1
        return num2 //n2
            

# 看不懂，通过100%
class Solution(object):
    def getMaxRepetitions(self, s1, n1, s2, n2):
        """
        :type s1: str
        :type n1: int
        :type s2: str
        :type n2: int
        :rtype: int
        """
        from bisect import bisect_right
        if n1==0:
            return 0
        l1,l2 = len(s1),len(s2)
        idx = {}
        f = []
        i = j = 0
        L1 = l1*n1
        while i<L1:
            while i<L1 and s1[i%l1]!=s2[j%l2]:
                i += 1
            if i<L1 and (j+1)%l2==0:
                if i%l1 not in idx:
                    idx[i%l1] = len(f)
                    f.append(i)
                else:
                    b = idx[i%l1]
                    a = len(idx)
                    rest = L1-f[b]
                    L = i-f[b]
                    k = bisect_right(f,rest%L+f[b]-1) if rest%L else b
                    # print a,b,rest,L
                    # print idx,f
                    ans = (rest//L)*(a-b)+k
                    return ans//n2
            i += 1
            j += 1
        return len(f)//n2


In [5]:
prices = [7,1,5,3,6,4]
n = len(prices)
dp = [[0]*2 for _ in range(n)]
dp[0][0]=0
dp[0][1]=-prices[0]
dp

[[0, -7], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]

In [18]:
def maxProfit(prices):
    n = len(prices)
    dp = [[0]*2 for _ in range(n)]
    dp[0][0] = 0
    dp[0][1] = -prices[0]
    for i in range(1, len(prices)):
        dp[i][0] = max(dp[i-1][0], dp[i-1][1]+prices[i])
        dp[i][1] = max(dp[i-1][1], -prices[i])
    print(dp)
    return dp[n-1][0]
            

# 740. 删除与获得点数

给定一个整数数组 nums ，你可以对它进行一些操作。

每次操作中，选择任意一个 nums[i] ，删除它并获得 nums[i] 的点数。之后，你必须删除每个等于 nums[i] - 1 或 nums[i] + 1 的元素。

开始你拥有 0 个点数。返回你能通过这些操作获得的最大点数。

示例 1:

输入: nums = [3, 4, 2]

输出: 6

解释: 

删除 4 来获得 4 个点数，因此 3 也被删除。

之后，删除 2 来获得 2 个点数。总共获得 6 个点数。

示例 2:

输入: nums = [2, 2, 3, 3, 3, 4]

输出: 9

解释: 

删除 3 来获得 3 个点数，接着要删除两个 2 和 4 。

之后，再次删除 3 获得 3 个点数，再次删除 3 获得 3 个点数。

总共获得 9 个点数。

注意:

nums的长度最大为20000。

每个整数nums[i]的大小都在[1, 10000]范围内。



In [44]:
# 动态规划：统计nums中数字出现的次数，问题就转换成与打家劫舍相同。

class Solution(object):
    def deleteAndEarn(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        num=collections.Counter(nums)
        
        dp=[0]*10001
        for i in range(10001):
            dp[i]=max(dp[i-1],dp[i-2]+i*num[i])
        return dp[10000]
